### Import necessary spark functions.

In [98]:
import org.apache.spark.sql.functions._ 
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification.{NaiveBayes,NaiveBayesModel,RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.ml._

https://kokes.github.io/blog/2018/05/19/spark-sane-csv-processing.html

In [20]:
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("PostId", DoubleType, true),
    StructField("PostCreationDate", StringType, true),
    StructField("OwnerUserId", DoubleType, true),
    StructField("OwnerCreationDate", StringType, true),
    StructField("ReputationAtPostCreation", DoubleType, true),
    StructField("OwnerUndeletedAnswerCountAtPostTime", DoubleType, true),
    StructField("Title", StringType, true),
    StructField("BodyMarkdown", StringType, true),
    StructField("Tag1", StringType, true),
    StructField("Tag2", StringType, true),
    StructField("Tag3", StringType, true),
    StructField("Tag4", StringType, true),
    StructField("Tag5", StringType, true),
    StructField("PostClosedDate", StringType, true),
    StructField("OpenStatus", StringType, true)
))

var df = spark.
    read.
    option("quote", "\"").
    option("escape", "\"").
    option("multiLine", "true").
    option("header", "true").
    schema(customSchema).
    csv("train-sample.csv")

customSchema = StructType(StructField(PostId,DoubleType,true), StructField(PostCreationDate,StringType,true), StructField(OwnerUserId,DoubleType,true), StructField(OwnerCreationDate,StringType,true), StructField(ReputationAtPostCreation,DoubleType,true), StructField(OwnerUndeletedAnswerCountAtPostTime,DoubleType,true), StructField(Title,StringType,true), StructField(BodyMarkdown,StringType,true), StructField(Tag1,StringType,true), StructField(Tag2,StringType,true), StructField(Tag3,StringType,true), StructField(Tag4,StringType,true), StructField(Tag5,StringType,true), StructField(PostClosedDate,StringType,true), StructField(OpenStatus,StringType,true))
df = [PostId: double, PostCre...


[PostId: double, PostCre...

### Read sample of training data set in from S3 bucket.

In [81]:
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("PostId", DoubleType, true),
    StructField("PostCreationDate", StringType, true),
    StructField("OwnerUserId", DoubleType, true),
    StructField("OwnerCreationDate", StringType, true),
    StructField("ReputationAtPostCreation", DoubleType, true),
    StructField("OwnerUndeletedAnswerCountAtPostTime", DoubleType, true),
    StructField("Title", StringType, true),
    StructField("BodyMarkdown", StringType, true),
    StructField("Tag1", StringType, true),
    StructField("Tag2", StringType, true),
    StructField("Tag3", StringType, true),
    StructField("Tag4", StringType, true),
    StructField("Tag5", StringType, true),
    StructField("PostClosedDate", StringType, true),
    StructField("OpenStatus", StringType, true)
))

var df = spark.
    read.
    option("quote", "\"").
    option("escape", "\"").
    option("multiLine", "true").
    option("header", "true").
    schema(customSchema).
    csv("s3://stackoverflow-kaggle/data/train-sample.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.types._
customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(PostId,DoubleType,true), StructField(PostCreationDate,StringType,true), StructField(OwnerUserId,DoubleType,true), StructField(OwnerCreationDate,StringType,true), StructField(ReputationAtPostCreation,DoubleType,true), StructField(OwnerUndeletedAnswerCountAtPostTime,DoubleType,true), StructField(Title,StringType,true), StructField(BodyMarkdown,StringType,true), StructField(Tag1,StringType,true), StructField(Tag2,StringType,true), StructField(Tag3,StringType,true), StructField(Tag4,StringType,true), StructField(Tag5,StringType,true), StructField(PostClosedDate,StringType,true), StructField(OpenStatus,StringType,true))
df: org.apache.spark.sql.DataFrame = [PostId: double, PostCreationDate: string ... 13 more fields]


In [21]:
df.show()

+-----------+-------------------+-----------+-------------------+------------------------+-----------------------------------+--------------------+--------------------+---------------+-----------------+-------------+------------------+----------+-------------------+--------------------+
|     PostId|   PostCreationDate|OwnerUserId|  OwnerCreationDate|ReputationAtPostCreation|OwnerUndeletedAnswerCountAtPostTime|               Title|        BodyMarkdown|           Tag1|             Tag2|         Tag3|              Tag4|      Tag5|     PostClosedDate|          OpenStatus|
+-----------+-------------------+-----------+-------------------+------------------------+-----------------------------------+--------------------+--------------------+---------------+-----------------+-------------+------------------+----------+-------------------+--------------------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+-----------+-------------------+-----------+-------------------+------------------------+------

The `OpenStatus` column seems to have an issue with trailing whitespace -- there should be a column ending pipe operator `|` in the printed table above.  Using `trim` didn't work, so we're taking a regex approach instead:

In [22]:
df = df.withColumn("OpenStatus", regexp_extract(col("OpenStatus"), "([\\w\\s]+\\w)", 1))

df = [PostId: double, PostCreationDate: string ... 13 more fields]


[PostId: double, PostCreationDate: string ... 13 more fields]

Get the percentage of null values for each column -- Tag2, Tag3, and Tag4 are the only columns with a significant amount of missing data.

In [23]:
df.
    select(df.columns.map(c => (sum(when(col(c).isNull || col(c) === "" || col(c).isNaN, 1).otherwise(0)) / df.count()).alias(c)): _*).
    show

+------+----------------+-----------+-----------------+------------------------+-----------------------------------+-----+------------+--------------------+-------------------+-------------------+------------------+------------------+--------------+----------+
|PostId|PostCreationDate|OwnerUserId|OwnerCreationDate|ReputationAtPostCreation|OwnerUndeletedAnswerCountAtPostTime|Title|BodyMarkdown|                Tag1|               Tag2|               Tag3|              Tag4|              Tag5|PostClosedDate|OpenStatus|
+------+----------------+-----------+-----------------+------------------------+-----------------------------------+-----+------------+--------------------+-------------------+-------------------+------------------+------------------+--------------+----------+
|   0.0|             0.0|        0.0|              0.0|                     0.0|                                0.0|  0.0|         0.0|7.129006501653929E-6|0.19408720200752824|0.45855195619938405|0.7172208281053952|0.

Fill the `Tag` fields with "unknown" so they can be processed as categorical features.

In [24]:
df = df.na.fill("unknown", Seq("Tag1", "Tag2", "Tag3", "Tag4", "Tag5"))

df = [PostId: double, PostCreationDate: string ... 13 more fields]


[PostId: double, PostCreationDate: string ... 13 more fields]

## Simple model: Naive Bayes with Tags as features

To start with, we'll take a simple approach - encode the Tag columns as categorical variables and predict `OpenStatus` using Random Forest. Spark's RF Classifier example will serve as a reference ([link](https://github.com/apache/spark/blob/master/examples/src/main/scala/org/apache/spark/examples/ml/RandomForestClassifierExample.scala)).

In [26]:
val df_simple = df.
    select(col("Tag1"), col("Tag2"), col("Tag3"), col("Tag4"), col("Tag5"), col("OpenStatus"))

df_simple = [Tag1: string, Tag2: string ... 4 more fields]


[Tag1: string, Tag2: string ... 4 more fields]

### EDA - frequency of `OpenStatus` and `TagX`

We see that the label `OpenStatus` has only 5 distinct values while each of the tags has over 5000 distinct values.

In [75]:
df_simple.
    columns.
    map(c => {
            val count = df_simple.select(c).distinct.count
            f"Distinct $c values: $count"
    }).
    foreach(println)

Distinct Tag1 values: 5212
Distinct Tag2 values: 9295
Distinct Tag3 values: 11083
Distinct Tag4 values: 10030
Distinct Tag5 values: 7607
Distinct OpenStatus values: 5


`OpenStatus` takes on the value `open` in exactly 50% of all cases.  The category `too localized` appears least often at 4.4% of occurrences.  

In [31]:
val status_counts = df_simple.
    groupBy("OpenStatus").
    count().
    sort(col("count").desc).
    withColumn("perc_of_total", lit(100) * col("count") / df_simple.count())

status_counts.show(numRows = 10)

+-------------------+-----+------------------+
|         OpenStatus|count|     perc_of_total|
+-------------------+-----+------------------+
|               open|70136|              50.0|
|not a real question|30789|21.949498117942284|
|          off topic|17530|12.497148397399338|
|   not constructive|15659|11.163311280939888|
|      too localized| 6158| 4.390042203718489|
+-------------------+-----+------------------+



status_counts = [OpenStatus: string, count: bigint ... 1 more field]


[OpenStatus: string, count: bigint ... 1 more field]

To cut down on the number of distinct `TagX` values, we'll try keeping the top 90% of observations.  The bottom 10% will be replaced with "other".  Below we count the number of distinct tags in the top 90% of observations for each `TagX` column.  `Tag2` and `Tag3` still have over 500 distinct values, but this is far less than the 10,000+ values they originally had.

In [56]:
def get_percentile_count(df: DataFrame, percentile: Double)(col_name: String): Double = {
    
    val cumsum_window = Window.
      orderBy(col("count").desc).
      rowsBetween(Window.unboundedPreceding, Window.currentRow)

    val total_window = Window.
        rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
    
    df.
        groupBy(col_name).
        count().
        orderBy(col("count").desc).
        withColumn("fracObs", sum(col("count")).over(cumsum_window) / sum(col("count")).over(total_window)).
        filter(col("fracObs") <= percentile).
        count()
}

Array("Tag1", "Tag2", "Tag3", "Tag4", "Tag5").
    map(get_percentile_count(df_simple, 0.9))

get_percentile_count: (df: org.apache.spark.sql.DataFrame, percentile: Double)(col_name: String)Double


Array(108.0, 689.0, 973.0, 350.0, 0.0)

### Pipeline for Random Forest

Since there are null `Tag1` values, we use `setHandleInvalid("keep")` so that they are indexed rather than dropped ([so_link](https://stackoverflow.com/a/36113473)).

We want to create a StringIndexer for each Tag column; rather than create 5 variables we'll take a functional approach.  Note that `setHandleInvalid` is set to "keep" so that the indexer adds new indexes when it sees new labels in data sets other than our current data set ([StackOverflow link](https://stackoverflow.com/a/43917703/11407644)).

In [100]:
val featureCols = Array[String]("Tag1", "Tag2", "Tag3", "Tag4", "Tag5")

val featureIndexers = featureCols.map { colName =>
    new StringIndexer().
        setInputCol(colName).
        setOutputCol("indexed" + colName).
        setHandleInvalid("keep").
        fit(df_simple)
}

featureCols = Array(Tag1, Tag2, Tag3, Tag4, Tag5)
featureIndexers = Array(strIdx_611619faad8a, strIdx_cb5e9c7716c2, strIdx_cfcab24b941c, strIdx_410970f4bcd3, strIdx_77761dfab371)


Array(strIdx_611619faad8a, strIdx_cb5e9c7716c2, strIdx_cfcab24b941c, strIdx_410970f4bcd3, strIdx_77761dfab371)

Spark ML models expect a feature vector to be the only predictor.  [`VectorAssembler`](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler) is a transformer that combines a list of columns into a single vector column.

In [102]:
val assembler = new VectorAssembler().
    setInputCols(featureCols.map{x => "indexed" + x}).
    setOutputCol("features").
    setHandleInvalid("keep")

assembler = vecAssembler_1d91282ebf85


vecAssembler_1d91282ebf85

Just as with the categorical features, we index the response.  Keeping with convention, the indexed response is called `indexedLabel` rather than `indexedOpenStatus`.

In [101]:
val labelIndexer = new StringIndexer().
    setInputCol("OpenStatus").
    setOutputCol("indexedLabel").
    setHandleInvalid("keep").
    fit(df_simple)

labelIndexer = strIdx_b539aaa6ac67


strIdx_b539aaa6ac67

Since the response is indexed, we need a way to transform the predicted response back to its original string value.  This inverse transformer is called [`IndexToString`](https://spark.apache.org/docs/latest/ml-features.html#indextostring):

In [103]:
val labelConverter = new IndexToString().
    setInputCol("prediction").
    setOutputCol("predictionLabel").
    setLabels(labelIndexer.labels)

labelConverter = idxToStr_aaba441fd44a


idxToStr_aaba441fd44a

Finally, we can specify our model, multinomial [NaiveBayes](https://spark.apache.org/docs/latest/mllib-naive-bayes.html):

In [104]:
val nb = new NaiveBayes().
    setLabelCol("label").
    setFeaturesCol("features").
    setSmoothing(1.0).
    setModelType("multinomial")

nb = nb_b39000a2c562


nb_b39000a2c562

The pipeline is constructed by combining the feature indexers built for each of the `TagX` variables with the vector assember, label index, and model itself.

In [109]:
val nb_pipeline = new Pipeline().
    setStages(featureIndexers ++ Array(assembler, labelIndexer, nb, labelConverter))

nb_pipeline = pipeline_300358b41c3d


pipeline_300358b41c3d

Finally we can specify our model, a [`RandomForestClassifer`](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier):

In [75]:
val rf = new RandomForestClassifier().
    setLabelCol("label").
    setFeaturesCol("features").
    setNumTrees(10).
    setMaxBins(12000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_9df6a58d40ea


In [76]:
val rf_pipeline = new Pipeline().
    setStages(featureIndexers ++ Array(assembler, labelIndexer, rf, labelConverter))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pipeline: org.apache.spark.ml.Pipeline = pipeline_b75699e47b54


### Fit Pipeline

Split into train and test sets.

In [106]:
val Array(train, test) = df_simple.randomSplit(Array(0.7, 0.3))

train = [Tag1: string, Tag2: string ... 4 more fields]
test = [Tag1: string, Tag2: string ... 4 more fields]


[Tag1: string, Tag2: string ... 4 more fields]

Fit the pipeline to the training set to create the model.

In [111]:
val model = nb_pipeline.fit(train)

+---------------+-----+--------------------+
|predictionLabel|label|            features|
+---------------+-----+--------------------+
|      off topic|  0.0|    (5,[0],[3312.0])|
|      off topic|  0.0|[75.0,552.0,619.0...|
|  too localized|  0.0|[75.0,285.0,504.0...|
|           open|  2.0|[75.0,342.0,10612...|
|      off topic|  0.0|(5,[0,1],[75.0,23...|
+---------------+-----+--------------------+
only showing top 5 rows



model = pipeline_300358b41c3d
predictions = [Tag1: string, Tag2: string ... 15 more fields]


[Tag1: string, Tag2: string ... 15 more fields]

Make predictions.

In [ ]:
val predictions = model.transform(test)
predictions.select("predictionLabel", "OpenStatus", "features").show(5)

### Evaluate Model

Set up [MulticlassClassificationEvaluator](https://spark.apache.org/docs/2.3.0/api/scala/index.html#org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator).

In [119]:
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("accuracy")

evaluator = mcEval_28b51846e9f0


mcEval_28b51846e9f0

Our accuracy was ~27%.  To get a sense for how good or bad this is, let's look at how well the model would have performed if it simply guessed the most common response (`Open`) each time.

In [120]:
val accuracy = evaluator.evaluate(predictions)

accuracy = 0.27433144919963903


0.27433144919963903

In [144]:
predictions.
    groupBy("OpenStatus").
    count().
    withColumn("perc_of_total", lit(100) * col("count") / lit(predictions.count())).
    show(5)

+-------------------+-----+------------------+
|         OpenStatus|count|     perc_of_total|
+-------------------+-----+------------------+
|not a real question| 9334|22.167862062413906|
|      too localized| 1842| 4.374673443214744|
|   not constructive| 4692|11.143304992162637|
|          off topic| 5274|12.525530803210943|
|               open|20964| 49.78862869899777|
+-------------------+-----+------------------+



If the model had guessed `open` for each test observation, the accuracy would have been 49.8% -- much higher than the 27% that NaiveBayes achieved.

In [137]:
predictions.
    select("predictionLabel", "OpenStatus").
    withColumn("success", when(col("predictionLabel") === col("OpenStatus"), 1).otherwise(0)).
    groupBy("OpenStatus").
    agg((sum(col("success")) / count("*")).alias("accurracy")).
    show(5)

+-------------------+--------------------+
|         OpenStatus|           accurracy|
+-------------------+--------------------+
|not a real question|0.030319262909792158|
|      too localized| 0.05863192182410423|
|   not constructive|0.017263427109974423|
|          off topic|   0.586841107318923|
|               open|  0.3808433505056287|
+-------------------+--------------------+



A look at class-specific performance:

### Conclusion